In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from sklearn.model_selection import train_test_split
import numpy as np


In [82]:
# Load the data using image_dataset_from_directory()
data_dir = "C:\\Users\\farha\\Desktop\\Py\\Pencil_shading"
image_size = (224, 224)
batch_size = 20
class_names_list = np.array(tf.io.gfile.listdir(data_dir)).tolist()

In [102]:
if class_names_list is not None and len(class_names) > 0:
    print("Available class names:")
    print(class_names_list)

Available class names:
['Crayons', 'Oil Pastel', 'Outline', 'Painting', 'Pencil shading']


In [103]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    class_names=class_names_list,
    validation_split=0.2,  # 80% training, 20% validation
    subset='training',
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)


Found 3061 files belonging to 5 classes.
Using 2449 files for training.


In [105]:
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    class_names=class_names_list,
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

Found 3061 files belonging to 5 classes.
Using 612 files for validation.


In [106]:
# Configure the data pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

In [107]:
# Use ImageDataGenerator for data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [108]:
# Apply data augmentation to the training dataset
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

In [109]:
# Split the data into training and testing sets using sklearn
X = []
y = []
for images, labels in train_data:
    X.append(images.numpy())
    y.append(labels.numpy())
X = np.concatenate(X)
y = np.concatenate(y)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [111]:
# Preprocess the image data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [112]:
# Load the Xception model
base_model = Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3)
)

In [113]:
# Freeze the base model layers
base_model.trainable = False

In [114]:
# Add custom classification head
inputs = tf.keras.Input(shape=(image_size[0], image_size[1], 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [115]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [2]:
model = tf.keras.models.load_model('xception_model.h5')

In [3]:
image_path = "C:\\Users\\farha\\Desktop\\test images\\p1.jpg"
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224,224))
img = tf.keras.preprocessing.image.img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)
predictions = model.predict(img)
class_indices = np.argmax(predictions, axis=1)
predicted_labels = [class_names_list[i] for i in class_indices]
#['Crayons', 'Oil Pastel', 'Outline', 'Painting', 'Pencil shading']
print(predicted_labels)

1/1 [==============================] - 7s 7s/step


NameError: name 'class_names_list' is not defined

In [ ]:
# # Train the model
# epochs = 10
# batch_size = 16
# model.fit(
#     X_train, y_train,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test)
# )

Epoch 1/10
123/123 [==============================] - ETA: 0s - loss: 1.1888 - accuracy: 0.5947

ResourceExhaustedError: Graph execution error:

SameWorkerRecvDone unable to allocate output tensor. Key: /job:localhost/replica:0/task:0/device:CPU:0;28a3d894238b4eb9;/job:localhost/replica:0/task:0/device:GPU:0;edge_242_IteratorGetNext;0:0
	 [[{{node IteratorGetNext/_2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_test_function_12137]

In [ ]:
# Save the model
model.save('xception_model.h5')
